# Kream Data 크롤러

In [37]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import numpy as np

pd.options.display.max_rows=100
pd.options.display.max_columns=100

## 크롬드라이버 열고, kream 로그인

In [38]:
# 크롬창 열기
# driver = webdriver.Chrome('./chromedriver')

# 윈도우 사용자
driver = webdriver.Chrome('.//chromedriver.exe')

C:\Users\goshd\AppData\Local\Temp/ipykernel_19384/2144316095.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('.//chromedriver.exe')


In [ ]:
# 크림 홈페이지 접속 후 로그인 미리 실행
# https://kream.co.kr/

## list312 불러오기

In [41]:
list104 = pd.read_csv('./DB/p_number/list104.csv')

In [42]:
list104

,number,transactions
0,28029,8.9만
1,28030,4.5만
2,41637,"3,217"
3,41193,1.3만
4,43374,"9,868"
...,...,...
99,37818,4
100,37770,35
101,37463,"1,058"
102,43208,1


## 크롤링 함수 만들기

In [43]:
# 스크롤 다운 함수 정의
import datetime
    
def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        itemlist = driver.find_element_by_class_name("price_body")
        driver.execute_script("arguments[0].scrollBy(0, -100)", itemlist)
        driver.execute_script("arguments[0].scrollBy(0, document.body.scrollHeight)", itemlist)
        time.sleep(1)
        if datetime.datetime.now() > end:
            break

In [44]:
# 크롤링 함수
def crawler(number, seconds):
    # 신발 선택
    url = 'https://kream.co.kr/products/{0}'.format(number)
    driver.get(url)
#     # 암시적 대기, 웹페이지 전체가 뜰때까지 대기
#     driver.implicitly_wait(5)
    # 명시적 대기, 특정 Xpath가 뜰때까지 대기
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="panel1"]/div/table/tbody/tr[1]/td[3]')))
    # 이름 뽑기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    item = soup.select('div.main_title_box > p.title')
    product_name = item[0].text
    # 컬럼 뽑기
    item = soup.select('dl.detail_product')
    cols = item[0].text.replace('\n', '').replace(' ','')
    r1 = cols.find('출시일')
    r2 = cols.find('컬러')
    r3 = cols.find('발매가')
    release_date = cols[r1+3:r2]
    color = cols[r2+2:r3]
    release_price = cols[r3+3:]
    # end_date
    end_date = soup.select('td.table_td.align_right')[1].text.replace('\n','').replace(' ','')
    # 체결 내역 더보기 클릭 (copy Xpath)
    driver.find_element_by_xpath('//*[@id="panel1"]/a').click()
    driver.find_element_by_xpath('//*[@id="panel1"]/div/div/div[1]/div/div[3]/a').click()
    # 체결 내역 더보기 스크롤 다운
    doScrollDown(seconds)
    # 데이터 추출
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    size_list = []
    price_list = []
    date_list = []
    for i in range(len(soup.select('div.body_list > div.list_txt'))):
        item = soup.select('div.body_list > div.list_txt')[i].text.replace('\n', '').replace(' ','')
        if i%3 == 0:
            size_list.append(item)
        elif i%3 == 1:
            price_list.append(item)
        elif i%3 == 2:
            date_list.append(item)
    df = pd.DataFrame({'size': size_list, 'price': price_list, 'date': date_list})
    df['product'] = product_name
    df['release_date'] = release_date
    df['color'] = color
    df['release_price'] = release_price
    file_name = product_name.replace(' ','_')
    df.to_csv('DB/{0}.csv'.format(file_name))
    if df['date'].max() == end_date :
        print('<<크롤링 완벽히 성공>>')
    else:
        print("@@@@@@@@스크롤 부족!! 이 상품 다시 크롤링하세요@@@@@@@@")
    print('{0} ~ {1} ({2})'.format(df['date'].min(), df['date'].max(), end_date))
    print(f'{file_name} 저장완료!')
    print('----'*10)

## 크롤링하기
- crawler함수
    - number는 상품번호
    - seconds는 스크롤내리는 초 설정 (승혁이는 아마 엄청 크게 잡아야할 것 같음)
- 설명


20/05/28 ~ 21/12/06 (21/12/06)<br>
Nike_x_Travis_Scott_SB_Dunk_Low_(Special_Box) 저장완료!<br>
18번째 완료

- 위와 같은 메시지가 나오는데 이걸 설명해줄게
    - 20/05/28 ~ 21/12/06 (21/12/06) 여기서 앞은 우리가 추출한 최소,최대 날짜고, (괄호)안에 들어가는거는 최대날짜와 일치해야 모든 스크롤을 잘 돌린 것임.
    - 저장이 완료되면 몇번째 상품인지도 나오니깐, 만약에 데이터가 너무 커서 n번째에 중단되었다면 다음번 실행때는 n번째부터 돌아가도록 하면됨.


In [46]:
# for i in range(len(list104['number'])):
#     number = list104['number'][i]
#     crawler(number, 0)
#     print(f'{i}번째 {number} 크롤링 완료')
#     print('===='*10)

for i in range(len(list104['number'])):
    if i >= 4:
        number = list104['number'][i]
        crawler(number, 10)
        print(f'{i}번째 {number} 크롤링 완료')
        print('===='*10)
    else:
        continue

C:\Users\goshd\AppData\Local\Temp/ipykernel_19384/1329951444.py:27: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="panel1"]/a').click()
C:\Users\goshd\AppData\Local\Temp/ipykernel_19384/1329951444.py:28: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//*[@id="panel1"]/div/div/div[1]/div/div[3]/a').click()


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=96.0.4664.110)
Stacktrace:
Backtrace:
	Ordinal0 [0x006D6903+2517251]
	Ordinal0 [0x0066F8E1+2095329]
	Ordinal0 [0x00572710+1058576]
	Ordinal0 [0x0059E324+1237796]
	Ordinal0 [0x00594037+1196087]
	Ordinal0 [0x005B64D3+1336531]
	Ordinal0 [0x00593A36+1194550]
	Ordinal0 [0x005B65BA+1336762]
	Ordinal0 [0x005C5BBF+1399743]
	Ordinal0 [0x005B639B+1336219]
	Ordinal0 [0x005927A7+1189799]
	Ordinal0 [0x00593609+1193481]
	GetHandleVerifier [0x00865904+1577972]
	GetHandleVerifier [0x00910B97+2279047]
	GetHandleVerifier [0x00766D09+534521]
	GetHandleVerifier [0x00765DB9+530601]
	Ordinal0 [0x00674FF9+2117625]
	Ordinal0 [0x006798A8+2136232]
	Ordinal0 [0x006799E2+2136546]
	Ordinal0 [0x00683541+2176321]
	BaseThreadInitThunk [0x76B26739+25]
	RtlGetFullPathName_UEx [0x77B08AFF+1215]
	RtlGetFullPathName_UEx [0x77B08ACD+1165]


#### 참고 문헌
- https://hello-bryan.tistory.com/194,  https://jcstory94.tistory.com/69  스크롤다운 설명 링크
- https://yganalyst.github.io/pythonic/memo_16_except/, https://medium.com/@peteryun/python-selenium%EC%9D%84-%ED%99%9C%EC%9A%A9%ED%95%9C-%ED%81%AC%EB%A1%A4%EB%9F%AC-%EB%A7%8C%EB%93%A4%EA%B8%B0-b055cefd1195 예외처리
- https://yeko90.tistory.com/39 셀레니움 암시적 대기